In [1]:
from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)

In [2]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user', 'movie', 'rating', 'timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
last_skywalker = np.array([0.98,0.9,-0.9])
user1 = np.array([0.9, 0.8, -0.6])
(user1*last_skywalker).sum()

2.1420000000000003

In [4]:
# The Dot Product
user1@last_skywalker.T

2.1420000000000003

In [5]:
casablanca = np.array([-0.99,-0.3,0.8])
user1@casablanca.T

-1.611

In [6]:
movies = pd.read_csv(path/'u.item', delimiter='|', encoding='latin-1',
                     usecols=(0,1), names=('movie', 'title'), header=None)
movies.head()
#usecols just returns the 2 columns

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [7]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [8]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

,user,title,rating
0,848,Indiana Jones and the Last Crusade (1989),5
1,87,Butch Cassidy and the Sundance Kid (1969),5
2,13,Alice in Wonderland (1951),4
3,378,"Three Musketeers, The (1993)",3
4,843,Sleeper (1973),2
5,395,Beavis and Butt-head Do America (1996),1
6,698,Back to the Future (1985),2
7,928,Citizen Kane (1941),5
8,336,Sense and Sensibility (1995),4
9,493,Willy Wonka and the Chocolate Factory (1971),3


In [9]:
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_factors=5
user_factors = torch.randn(n_users, n_factors)
movie_factors = torch.randn(n_movies, n_factors)

In [10]:
# NN models can't look up index 
one_hot_3 = one_hot(3, n_users).float()
user_factors.T @ one_hot_3

tensor([-0.7793,  0.4836,  0.0628, -0.0196, -0.2048])

In [11]:
user_factors[3]

tensor([-0.7793,  0.4836,  0.0628, -0.0196, -0.2048])

In [12]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors):
        self.user_factors=Embedding(n_users, n_factors)
        self.movie_factors=Embedding(n_movies, n_factors)
        
    def forward(self, x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        return (users*movies).sum(dim=1)

    

In [13]:
x,y=dls.one_batch()
x.shape

torch.Size([64, 2])

In [14]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())

In [15]:
learn.fit_one_cycle(5, 5e-3)

RuntimeError: Exception occured in `TrainEvalCallback` when calling event `before_fit`:
	CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.